Librerias necesarias


In [9]:
!pip install flask==0.12.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
!pip install flask-ngrok==0.0.25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# BLockChain con Python


In [11]:
# Importacion de las librerias
import datetime
import hashlib
import json
from flask import Flask, jsonify
from flask_ngrok import run_with_ngrok

Creación de una clase Blockcain que contega todos los métodos esenciales:


*   Creación de un nuevo bloque
*   Obtención del hash del bloque
*   Protocolo de consenso Pow
*   Genereación del hash de un bloque
*   Verificacion de la validez de la BlockChain






In [12]:
from typing_extensions import NoReturn
# Creación de la clase BlockChain
class Blockchain:
  def _init_(self):
    """Constructor de la clase Blockchain"""
    self.chain = []
    self.create_block(proof=1, previous_hash= '0')

  def create_block(self, proof, previous_hash):
    """Creacion de un nuevo bloque
      Argumentos:
      - proof: Hash del bloque actual
      - previous_hash: 
      Returns:
      - block: Nuevo bloque creado
    """
    block = {
        'index'         :len(self.chain)+1,
        'timestamp'     :str(datetime.datetime.now()),
        'proof'         :proof,
        'previous_hash' : previous_hash
    }
    self.chain.append(block)
    return block
  
  def get_previous_block(self):
    """ Obtencion del bloque previo de la blockChain   """
    return self.chain[-1]


def proof_of_work(self, previous_hash):
  """ Protocolo de consenso
    Argumentos:
      - previous_hash: NOUNCE del bloque previo
      Returns:
      - new_proof: Devolución del nuevo nounce obtenido con Pow
  """
  new_proof = 1
  check_proof = False
  while check_proof is False:
    # Se intenta que los hashes no coincidan
    hash_operation = hashlib.sha256(
        str(
            new_proof**2 - previous_hash**2
        ).encode()
    ).hexdigest()
    if hash_operation[:4] == '0000':
      check_proof=True
    else:
        new_proof += 1

  return new_proof

def hash(self, block):
  """
    Calculo del hash de un bloque.
    Arguments:
    - Block: identifica el bloque de la blockchain

    Returns:
    - hash del bloque 
  """
  encode_block = json.dump(block, sort_key = True).encode() # convert BLOCK
  hash_block = hashlib.sha256(encode_block).hexdigest()
  return hash_block

def is_chain_valid(self,chain):
  """
    Determina si la Blockchain es válida

    Arguments:
    -
  """
  previous_block = chain[0]
  block_index = 1
  while block_index < len(chain):
      block = chain[block_index]
      if block['previous_hash'] != self.hash(previous_block):
          return False
      previous_proof = previous_block['proof']
      proof = block['proof']
      hash_operation = hashlib.sha256(str(proof**2 - previous_proof**2).encode()).hexdigest()
      if hash_operation[:4] != '0000':
          return False
      previous_block = block
      block_index += 1
  
  return True


Aplicación Web accesible vía en modo de REST API. Mediante llamadas HTTP al REST API podemos establecer uan comunicación con el uso del módulo de Flask. 

Llamadas a realizar vía REST API:
* Minar bloques: **mine_block()**
* Obtención de la Blockchain: **get_chain()**
* Comprobar estado de la Blockchain: **is_valid()**

In [13]:
# Crear una aplicación web
# Ejecución de la app con Notebook
app = Flask(__name__)
run_with_ngrok(app)  

# Si se obtiene un error 500, actualizar flask y ejecutar la siguiente línea
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False

# Creación de la Blockchain
blockchain = Blockchain()


@app.route('/mine_block', methods=['GET'])
def mine_block():
  """ Minado de un nuevo bloque """

  previous_block  = blockchain.get_previous_block() #ultimo bloque de la cadena
  previous_proof  = previous_block['proof'] # nonce del bloque previo
  proof           = blockchain.proof_of_work(previous_proof) #nuevo nonce
  previous_hash   = blockchain.hash(previous_block) # hash previo
  block           = blockchain.create_block(proof, previous_hash)
  response = {'message'       : '¡Enhorabuena, has minado un nuevo bloque!', 
              'index'         : block['index'],
              'timestamp'     : block['timestamp'],
              'proof'         : block['proof'],
              'previous_hash' : block['previous_hash']}
  return jsonify(response), 200

@app.route('/get_chain', methods=['GET'])
def get_chain():
  """ Obtención de la Blockchain """
  response = {'chain'   : blockchain.chain, 
              'length'  : len(blockchain.chain)}
  return jsonify(response), 200

@app.route('/is_valid', methods = ['GET'])
def is_valid():
  """ Comprobación si la Blockchain es válida """

  is_valid = blockchain.is_chain_valid(blockchain.chain)
  if is_valid:
      response = {'message' : '¡La cadena de bloques es válida!'}
  else:
      response = {'message' : '¡La cadena de bloques NO es válida!'}
  return jsonify(response), 200

In [15]:
# Ejecución de la app con Google Colab
app.run()

# Ejecución externa a Google colab
# app.run(host = '0.0.0.0', port = 5000)

INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://cb59-34-125-184-217.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [24/Jan/2023 17:11:19] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2023 17:11:20] "GET /favicon.ico HTTP/1.1" 404 -
